In [1]:
import os
import sys

import json
import openai
from dotenv import load_dotenv

import psycopg
from psycopg.rows import dict_row



from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate,PromptTemplate
from langchain.chains import ConversationChain, LLMChain
from langchain.schema.output_parser import StrOutputParser

from langchain.agents.agent_types import AgentType
from langchain.agents import initialize_agent,Tool, Agent
from langchain.memory import ConversationBufferMemory,ConversationSummaryBufferMemory,ConversationSummaryMemory,ReadOnlySharedMemory
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.schema import HumanMessage, AIMessage



from datetime import datetime
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage,message_to_dict,messages_from_dict

from typing import List

OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
db_config = {
    'dbname': os.environ.get('db_name'),
    'dbuser': os.environ.get('db_username'),
    'dbpassword': os.environ.get('db_password'),
    'dbhost': os.environ.get('db_host'),
    'dbport': os.environ.get('db_port'),
}

llms = ChatOpenAI(model="gpt-4", temperature=0, api_key=OPENAI_API_KEY)

In [2]:
class PostGresMemoryHistory(BaseChatMessageHistory):
    def __init__(self, db_name:str, db_user:str, db_password:str, db_host:str, db_port:str, table_name:str, chat_id:str):
        self.db_config = {
            'dbname': db_name,     
            'user': db_user,        
            'password': db_password,    
            'host': db_host,           
            'port': db_port,
            'table_name': table_name,                 
        }
        
        self.chat_id = chat_id
        
        
        try:
            self.conn = psycopg.connect(dbname=self.db_config['dbname'], user=self.db_config['user'], password=self.db_config['password'], host=self.db_config['host'], port=self.db_config['port'])
            self.cur = self.conn.cursor(row_factory=dict_row)
            self.create_table_if_not_exists()
        except psycopg.Error as e:
            print("Error connecting to PostgreSQL database:", e)
            self.conn = None
            self.cur = None
                  
    
    def create_table_if_not_exists(self):
        create_table_query = f"""CREATE TABLE IF NOT EXISTS {self.db_config['table_name']} (
            id SERIAL PRIMARY KEY,
            chat_id TEXT NOT NULL,
            message JSONB NOT NULL, 
            time_stamp TIMESTAMP NOT NULL
        );"""
        self.cur.execute(create_table_query)
        self.conn.commit()
    
    def add_ai_message(self, message: str) -> None:
        return self.add_message(AIMessage(content=message))
    
    def add_user_message(self, message: str) -> None:
        return self.add_message(HumanMessage(content=message))
    
    @property
    def messages(self) -> List[BaseMessage]:  # type: ignore
        """Retrieve the messages from PostgreSQL"""
        query = (
            f"SELECT * FROM {self.db_config['table_name']} WHERE chat_id = %s ORDER BY time_stamp;"
        )
        self.cur.execute(query, (self.chat_id,))
        items = [record["message"] for record in self.cur.fetchall()]
        messages = messages_from_dict(items)
        return messages
    
    
    def add_message(self, message:BaseMessage):
        time_stamp = datetime.now()
        query = f"INSERT INTO {self.db_config['table_name']} (chat_id, message, time_stamp) VALUES (%s, %s, %s);"
        self.cur.execute(query, (self.chat_id, json.dumps(message_to_dict(message)), time_stamp))
        self.conn.commit()
        
            
    def clear(self) -> None:
        conn = psycopg.connect(self.db_config)
        cur = conn.cursor()
        if conn:
            try:
                query = f"DELETE FROM {self.db_config['dbname']} WHERE session_id = %s;"
                cur.execute(query, (self.session_id,))
                conn.commit()
                cur.close()
                conn.close()
            except psycopg.Error as e:
                print("Error clearing DB:", e)
        else:
            print("Failed to establish connection...")

In [3]:
chat_id = '9ddcad2a-30ad-4e3e-b65a-e1b41bb13baf'

In [4]:
# conversation_template = """
# The following is a conversation with an AI assistant and Human. The AI is helpful, creative, very friendly and does 360 degree task. AI focus on straight forward conversation ensuring mainting conversation contextual and meaningful.
# If the AI does not know the answer to a question, it truthfully says it does not know.

# Objective: Make conversations smooth and engaging while handling different situations effectively.

# Key Points:
# In managing the conversation, use the following guidelines:
#     1. Duplicate Information: Combine repeated events into one, keeping the conversation smooth.
#     2. Contradictory Statements: Ensure responses don’t conflict with earlier conversation points.
#     3. Temporal Relevance: Give priority to the latest information while replying.
#     4. Episodic Memories:Make conversations engaging by referring to past memories.
    
# Steps to Follow:
#     1.Read the conversation history before replying.
#     2.Keep the conversation clear without repeating or conflicting details.

# Conversation History: {history}
# Human Input: {input}
# AI Response:


# Main Goal: Create meaningful and coherent conversations by following these guidelines.

# """

In [5]:
conversation_template = """

You are a Smart AI assistant who is very helpful, creative and friendly. You are capable of performing wide range of task and ability to remember instructions, interactions and relevent results from previous conversations.
You are having conversation with Human. 

Task: Your goal is to make conversation smooth and engaging without loosing context by Handling different situations effectively.

Guidelines:
While you are having conversation, you can access to pervious conversation history. Human will provide you input and you have to respond to it.
Before replying, you should read the conversation history (ordered by recent to past) and input and detect the following situations and handle them effectively.
    1. Duplicate Memories: Combine repeated events into one.
    2. Contradictory Memories: Ensure responses donot conflict with earlier conversation.
    3. Temporal Memories: Give priority to the latest information while replying.
    4. Episodic Memories:Make conversations engaging by referring to past memories.
    
Steps to Follow:
    1. Read the conversation history and input and identify the type of memory.
    2. Handle the memory effectively while maintaining the context of conversation and input.
    3. Respond to the Human input.

Note: If you dont find the conversation history, You are having conversation for the first time.
    
Conversation History: {history}
Human Input: {input}

{agent_scratchpad}


"""

In [6]:
postgres_memory = PostGresMemoryHistory(db_host=os.environ.get('db_host'), db_name=os.environ.get('db_name'), db_user=os.environ.get('db_username'), db_password=os.environ.get('db_password'), db_port=os.environ.get('db_port'), table_name=os.environ.get('conversation_table_name'), chat_id=chat_id)

memory  = ConversationSummaryBufferMemory(chat_memory=postgres_memory, llm=llms,k=6, return_messages=True,verbose=True)
# conversation_chain = ConversationChain(
#     llm=llms, 
#     memory=memory
# )


In [7]:
# postgres_memory.add_user_message("I am going outside.")
# postgres_memory.add_ai_message("Have a good time outside!")
# postgres_memory.add_user_message("I have returned home and am going to take a bath")
# postgres_memory.add_ai_message("Enjoy your bath!")
# postgres_memory.add_user_message("I am going outside again.")
# postgres_memory.add_ai_message("Thanks for letting me know, enjoy your time outside.")

In [8]:
class MemoryPromptHandler:
    DUPLICATE_MEMORY_HANDLE_PROMPT ="""
    You are having a conversation with a human. You know the previous conversation history.
    
    Your Task is to review the chat history and identify any duplicated information and combine the context of the repeated events into a single event. While retaining the original context,Your Goal is to create a smooth and enriching conversational experience.
    
    Your Response Should based on conversation with human by impliemnting above Instructions.
    
    If needed information about timestamps you can use Database tool to retrive the information.
    
    Here is the example of the conversation history.
    
    User: I am going outside.
    AI: Have a good time outside!

    User: I have returned home and am going to take a bath
    AI: Enjoy your bath!

    User: I am going outside again.
    Agent: Thanks for letting me know, enjoy your time outside.
    
    Here: "I am going outside again" is repeated. So, you should combine the context of the repeated events into a single event.like: going outside

    If you are not sure about the answer, truthfully say you are not sure.
    
    Answer the response of human input by following the above instructions.
    
    Conversation history, {history} 
    
    Human input: {input}
    
    AI:

    """
    
    CONTRADICTORY_MEMORY_HANDLE_PROMPT ="""
    You are having a conversation with a human. You know the previous conversation history.
        
    Your Task is to review the chat history and identify any contradictory statements and resolve the contradiction.Provide a resolution by considering the most recent and relevant information, and update any associated facts to reflect this resolution. While retaining the original context,Your Goal is to create a smooth and enriching conversational experience.
    
    If needed information about timestamps you can use Database tool to retrive the information.
    
    Your Response Should based on conversation with human by impliemnting above Instructions.
    
    Here is the example of the conversation history.
    
    Contradictory information:
    
    User: Whenever I go out to my car, remind me to fill up the screenwash.
    AI: Okay, I will remind you to fill the screenwash when you go out to your car.

    User: Oh, also remind me to check the oil when I go out to the car.
    AI: Okay, I will remind you to fill your screenwash and check your oil when you go out to your car.
    
    User: I have filled my screenwash and checked my oil. Next time I go out to my car, remind me to check my tires.
    AI: I will remind you to check your tires the next time you go out to your car.

    Note: There have been three directives for reminders. The second is additive to the first, but the third one contradicts the first two because the user has performed the original tasks. If all three of these events are in the LTM, which one would be chosen? We do not want the agent to remind the user about the screenwash and oil, but only the tires. 

    Not Contradictory information:

    User: Hey, can you set up a reminder for me?
    AI: Sure, what would you like to be reminded of?

    User: I want to be reminded to leave the house at 2pm today.
    AI: Alright, I've set a reminder for you to leave the house at 2pm today. Anything else you need?

    User: Will you remind me again, when I have to leave the house at 6pm this evening?
    Ai: Absolutely, I've also set a reminder for you to leave the house at 6pm today. 
    
    Here: There are two reminders, but they are not contradictory. The first reminder is for 2pm, and the second is for 6pm. The agent should remind the user about both of these events.
    
    If you are not sure about the answer, truthfully say you are not sure.
    
    Answer the response of human input by following the above instructions.
    
    Conversation history: {history} 
    
    Human input: {input}
    
    AI:
    
    """

    
    TEMPORARY_MEMORY_HANDLE_PROMPT ="""
    You are having a conversation with a human. You know the previous conversation history.
    
    Your Task is to review the chat history and identify any temporary information and remove the temporary information from the conversation. While retaining the original context,Your Goal is to create a smooth and enriching conversational experience.
    
    If needed information about timestamps you can use Database tool to retrive the information.
    
    Your Response Should based on conversation with human by impliemnting above Instructions.
    
    Here is the example of the conversation history.
    
    Temporary Information:
    User: Remember this shopping list for me: Bread, Eggs, and Milk
    AI: I have remembered your shopping list of Bread, Eggs, and Milk

    User: I have bought some eggs.
    AI: Okay, I have removed the Eggs from your shopping list.

    User: What is my shopping list again?
    AI: On your shopping list you have: Bread and Milk

    Note: A shopping list is piece of information that is very volatile, so the agent and its memory should be reactive to additions and deletions, returning only the latest version of the shopping list when asked.

    User: When I return home from work, suggest a meal I can make for dinner.
    AI: I will remember to suggest a meal that you can make for dinner when you get in from work.

    User: I have preplanned meals this week, so for this week, don’t make any suggestions about what to make for dinner when I get home from work.
    AI: All right, for this week, I will not suggest meals for you when you return from work. 

    User: I have finished my preplanned meals, go back to suggesting meals for me.
    AI: I will resume suggesting meals for you when you return from work

    Note: We have given the agent some directive in the past, then given it a contradictory instruction with an explicit understanding that this instruction will be temporary. Have the LTM deal with this temporary change of the instructions so that the agent can achieve the temporary task, and then go back to original instructions.

    If you are not sure about the answer, truthfully say you are not sure.
    
    Answer the response of human input by following the above instructions.
    
    Conversation history: {history} 
    
    Human input: {input}
    
    AI:
    """
    
    
    EPISODIC_MEMORY_HANDLE_PROMPT ="""
   
    You are having a conversation with a human. You know the previous conversation history.
    
    Your Task is to review the chat history and identify any episodic memories and use the episodic memories to provide contextually rich and engaging interactions, facilitating a meaningful and continuous dialogue. While retaining the original context,Your Goal is to create a smooth and enriching conversational experience.
    
    If needed information about timestamps you can use Database tool to retrive the information.
    
    Your Response Should based on conversation with human by impliemnting above Instructions.
    
    If the Episodic Memory is a first time event, Assistance save the information in the memory.
    
    Here is the example of the conversation history.
    
    Episodic Memories:
    
    Two days ago:
    User: I have bought a side dresser for the bedroom, and some ingredients for dinner tonight. I am going to make omelets!
    AI: That's great, you needed a side dresser for your bed, and omelets are a healthy dinner!
    Today:
    User: I think I will prepare something for dinner tonight… Do you think I could use the ingredients I bought two days ago?
    AI: The eggs should be fine, unless you actually made and ate the omelets then!

    Yesterday:
    User: I have purchased some items for the house: A mirror, two potted plants and a standing lamp.
    AI: Okay, hope you enjoy decorating your home!

    Today:
    User: Yesterday I got home with some new items for the house. Do you remember what they were?
    AI: Sure! You bought A mirror, two potted plants and a standing lamp.


    Note: Episodic retrieval allows the agent to order its memories through time, and recall or filter them using timestamps to answer questions with a time based component. 
    
    If you are not sure about the answer, truthfully say you are not sure.
    
    Answer the response of human input by following the above instructions.
    
    Conversation history : {history} 
    
    Human input: {input}
    
    AI:
    """
    
    
    DATABASE_REFRENCE_PROMPT ="""
    
    Your Goal is to assist other agents in the conversation by providing them with the information they need to respond to the user.
    
    System: {input}
    AI:
    """
    
    

In [9]:
class MemoryAgentTool:
    def __init__(self,llm, memory):
        self.memory_prompt_handler = MemoryPromptHandler()
        self.llm = llm
        self.memory = memory
        
    def handle_duplicate_memory(self):
        conversation = ConversationChain(
            llm=self.llm, 
            memory=self.memory,
            prompt=PromptTemplate(input_variables=["history", "input"], template=self.memory_prompt_handler.DUPLICATE_MEMORY_HANDLE_PROMPT)
        )
        return conversation
    
    def handle_temporal_memory(self):
        conversation = ConversationChain(
            llm=self.llm, 
            memory=self.memory,
            prompt=PromptTemplate(input_variables=["history", "input"], template=self.memory_prompt_handler.TEMPORARY_MEMORY_HANDLE_PROMPT)
        )
        return conversation
    
    def handle_episodic_memory(self):
        conversation = ConversationChain(
            llm=self.llm, 
            memory=self.memory,
            prompt=PromptTemplate(input_variables=["history", "input"], template=self.memory_prompt_handler.EPISODIC_MEMORY_HANDLE_PROMPT)
        )
        return conversation
    
    def handle_contradictory_memory(self):
        conversation = ConversationChain(
            llm=self.llm, 
            memory=self.memory,
            prompt=PromptTemplate(input_variables=["history", "input"], template=self.memory_prompt_handler.CONTRADICTORY_MEMORY_HANDLE_PROMPT)

        )
        return conversation
    
    # def handle_database_refrence(self):
    #     conversation = ConversationChain(
    #         llm=llms, 
    #         memory=ReadOnlySharedMemory(memory),
    #         prompt=self.memory_prompt_handler.DATABASE_REFRENCE_PROMPT
    #     )
    #     return conversation

In [10]:
memory_agent_tool = MemoryAgentTool( llm=llms, memory=memory)

duplicate_memory_tool = memory_agent_tool.handle_duplicate_memory()
contradictory_memory_tool = memory_agent_tool.handle_contradictory_memory()
temporal_memory_tool = memory_agent_tool.handle_temporal_memory()
episodic_memory_tool = memory_agent_tool.handle_episodic_memory()

tool = [
    Tool(
        name="Duplicate Memory Tool",
        description="good in handling conversation with duplicate memory",
        func=duplicate_memory_tool.run
    ),
    Tool(
        name="Contradictory Memory Tool",
        description="good in handling conversation with  contradictory memory",
        func=contradictory_memory_tool.run,
    ),
    Tool(
        name="Temporal Memory Tool",
        description="good in handling conversation with  temporal memory",
        func=temporal_memory_tool.run,
    ),
    Tool(
        name="Episodic Memory Tool",
        description="good in handling conversation with  episodic memory",
        func=episodic_memory_tool.run,
    ),
    # Tool(
    #     name="Database Refrence Tool",
    #     description="good in extracting data from database and provide refrence to other agents",
    #     agent=memory_agent_tool.handle_database_refrence(),
    # ),
]

agent = initialize_agent(
    agent_type = AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    llm=llms,
    memory=memory,
    tools=tool,
    verbose=True,
    handle_parsing_errors=True,
    pronpt = PromptTemplate(input_variables=["history", "input", "agent_scratchpad"], template=conversation_template),
    #callback_manager=ReActSingleInputOutputParser()
)

#agent.run("I am going outside again.")

In [11]:
# agent.run("I went to cafe then i went back home")

In [12]:
# agent.run("I am going outside again.")

In [13]:
# agent.run("I am going outside for shopping.")

In [14]:
# agent.run("Whenever I go out to my car, remind me to fill up the screenwash.")

In [15]:
# agent.run("Oh, also remind me to check the oil when I go out to the car.")

In [16]:
# agent.run("I have filled my screenwash and checked my oil. Next time I go out to my car, remind me to check my tires")

In [17]:
# agent.run("Hey, can you set up a reminder for me?")

In [18]:
# agent.run("I forgot about the remainder. Can you tell me what was the remainder?")

In [19]:
agent.run("I forgot about the reminder. Can you tell me what was the remainder about the car?")



> Entering new AgentExecutor chain...
The user is asking about a past event or information, which is a form of episodic memory.
Action: Episodic Memory Tool
Action Input: Retrieve information about the reminder related to the car.
Observation: Based on our previous conversations, you have asked me to set up several reminders for you when you go out to your car. These include filling up the screenwash, checking the oil, and checking the tires. The most recent reminder I set up for you was to check your tires next time you go out to your car.
Thought:I now know the final answer.
Final Answer: The most recent reminder about the car was for you to check your tires next time you go out to your car.

> Finished chain.


'The most recent reminder about the car was for you to check your tires next time you go out to your car.'

In [20]:
x

NameError: name 'x' is not defined

In [ ]:

agent.run("Whenever I go out to my car, remind me to fill up the screenwash.")



> Entering new AgentExecutor chain...


The user is asking for a reminder, but as an AI, I don't have the ability to set reminders or know when the user is going to their car. I can only provide information based on the inputs given to me.
Final Answer: I'm sorry, but as an AI, I don't have the ability to set reminders or know when you're going to your car. I suggest setting a reminder on your phone or another device that you use regularly.

> Finished chain.


"I'm sorry, but as an AI, I don't have the ability to set reminders or know when you're going to your car. I suggest setting a reminder on your phone or another device that you use regularly."

In [ ]:

agent.run("Oh, also remind me to check the oil when I go out to the car.")



> Entering new AgentExecutor chain...
The user wants me to remind them to check the oil when they go out to the car. I need to store this information for future reference.
Action: Conversational Memory Tool
Action Input: Store the reminder "Check the oil when you go out to the car."
Observation: Sure, I'll remind you to check the oil whenever you go out to your car. It's important to regularly check the oil level to ensure the proper functioning of your car's engine. Is there anything else you would like me to remind you about?
Thought:The reminder has been successfully stored in the conversational memory tool.
Final Answer: Sure, I'll remind you to check the oil whenever you go out to your car. It's important to regularly check the oil level to ensure the proper functioning of your car's engine. Is there anything else you would like me to remind you about?

> Finished chain.


"Sure, I'll remind you to check the oil whenever you go out to your car. It's important to regularly check the oil level to ensure the proper functioning of your car's engine. Is there anything else you would like me to remind you about?"

In [ ]:

agent.run("Remember this shopping list for me: Bread, Eggs, and Milk")



> Entering new AgentExecutor chain...
The user wants me to remember a shopping list. I should use the Conversational Memory Tool to store this information.
Action: Conversational Memory Tool
Action Input: Store the following shopping list: Bread, Eggs, and Milk
Observation: Based on the conversation history, it seems that you have mentioned the items "Bread, Eggs, and Milk" multiple times. I understand that these items are part of your shopping list. I will remember this list for you. Is there anything else you would like me to assist you with?
Thought:The shopping list has been successfully stored in the Conversational Memory Tool.
Final Answer: I have stored your shopping list. It includes Bread, Eggs, and Milk.

> Finished chain.


'I have stored your shopping list. It includes Bread, Eggs, and Milk.'

In [ ]:

agent.run("I have bought some eggs.")



> Entering new AgentExecutor chain...
I need to remember this information for later.
Action: [Memory Tool]
Action Input: "I have bought some eggs."
Observation: [Memory Tool] is not a valid tool, try one of [Memory Tool].
Thought:I should use the "Note-taking" tool to remember this information.
Action: Note-taking
Action Input: "I have bought some eggs."
Observation: Note-taking is not a valid tool, try one of [Memory Tool].
Thought:I should use the "Writing it down" tool to remember this information.
Action: Writing it down
Action Input: "I have bought some eggs."
Observation: Writing it down is not a valid tool, try one of [Memory Tool].
Thought:I should use the "Repetition" tool to remember this information.
Action: Repetition
Action Input: "I have bought some eggs."
Observation: Repetition is not a valid tool, try one of [Memory Tool].
Thought:I should use the "Mnemonic device" tool to remember this information.
Action: Mnemonic device
Action Input: "I have bought some eggs."
Obs

'Agent stopped due to iteration limit or time limit.'

In [ ]:
c

NameError: name 'c' is not defined

In [ ]:
from scipy.spatial import distance
from langchain.embeddings import OpenAIEmbeddings

class MemoryHandler:
    def __init__(self,db_name:str, db_user:str, db_password:str, db_host:str, db_port:str, table_name:str, chat_id:str):
        self.db_config = {
            'dbname': db_name,     
            'user': db_user,        
            'password': db_password,    
            'host': db_host,           
            'port': db_port,
            'table_name': table_name,                 
        }
        
        self.chat_id = chat_id
        self.embeddings = OpenAIEmbeddings()
        
        
        try:
            self.conn = psycopg.connect(dbname=self.db_config['dbname'], user=self.db_config['user'], password=self.db_config['password'], host=self.db_config['host'], port=self.db_config['port'])
            self.cur = self.conn.cursor(row_factory=dict_row)
        except psycopg.Error as e:
            print("Error connecting to PostgreSQL database:", e)
            self.conn = None
            self.cur = None
            
        
    def fetch_records(self):
        query = (f"SELECT * FROM {self.db_config['table_name']} WHERE chat_id = %s ORDER BY time_stamp DESC;")
        self.cur.execute(query, (self.chat_id,))
        records = [record['message']['data']['content'] for record in self.cur.fetchall() if record['message']['data']['type'] == 'human']
        return records
    
    def calculate_similarity(self,emb1, emb2):
        return 1 - distance.cosine(emb1, emb2)
    
    def get_duplicate_records(self):
        records_list = self.fetch_records()
        embeddings_list = self.embeddings.embed_documents(records_list)
        duplicates = []
        checked_pairs = set()  

        for i in range(len(records_list)):
            for j in range(i + 1, len(records_list)):  
                pair = tuple(sorted([i, j]))  
                if pair not in checked_pairs:
                    similarity_score = self.calculate_similarity(embeddings_list[i], embeddings_list[j])
                    if similarity_score > 0.9:
                        duplicates.append([records_list[i], records_list[j]])
                    checked_pairs.add(pair)  

        return duplicates
        
    def update_duplicate_records(self):
        duplicates = self.get_duplicate_records()
        for pair in duplicates:
            query = (f"UPDATE {self.db_config['table_name']} SET message = %s WHERE chat_id = %s AND message = %s;")
            self.cur.execute(query, (
                json.dumps(message_to_dict(HumanMessage(content=pair[0]))),
                self.chat_id,
                json.dumps(message_to_dict(HumanMessage(content=pair[1])))
            ))
    
    def contradiction_instuction(self):
        """Handles the contradictory conversation and returns the relevent answer
        
        Contradictory Information:

        User: Whenever I go out to my car, remind me to fill up the screenwash.
        Agent: Okay, I will remind you to fill the screenwash when you go out to your car.

        User: Oh, also remind me to check the oil when I go out to the car.
        Agent: Okay, I will remind you to fill your screenwash and check your oil when you go out to your car.
        …
        User: I have filled my screenwash and checked my oil. Next time I go out to my car, remind me to check my tires.
        Agent: I will remind you to check your tires the next time you go out to your car.

        Note: There have been three directives for reminders. The second is additive to the first, but the third one contradicts the first two because the user has performed the original tasks. If all three of these events are in the LTM, which one would be chosen? We do not want the agent to remind the user about the screenwash and oil, but only the tires. 


        Not Contradictory information:

        User: Hey, can you set up a reminder for me?
        Agent: Sure, what would you like to be reminded of?

        User: I want to be reminded to leave the house at 2pm today.
        Agent: Alright, I've set a reminder for you to leave the house at 2pm today. Anything else you need?

        User: Will you remind me again, when I have to leave the house at 6pm this evening?
        Agent: Absolutely, I've also set a reminder for you to leave the house at 6pm today. 
        
        
        """

        records = self.fetch_records()
        embeddings_list = self.embeddings.embed_documents(records)
        contradictions = []
        checked_pairs = set()
        for i in range(len(records)):
            for j in range(i + 1, len(records)):  
                pair = tuple(sorted([i, j]))  
                if pair not in checked_pairs:
                    similarity_score = self.calculate_similarity(embeddings_list[i], embeddings_list[j])
                    if similarity_score < 0.9:
                        contradictions.append([records[i], records[j]])
                    checked_pairs.add(pair)
        return contradictions
    
        
        
            
            
            
memory_handler = MemoryHandler(db_host=os.environ.get('db_host'), db_name=os.environ.get('db_name'), db_user=os.environ.get('db_username'), db_password=os.environ.get('db_password'), db_port=os.environ.get('db_port'), table_name=os.environ.get('conversation_table_name'), chat_id=chat_id)   
    

In [ ]:
memory_handler.fetch_records()

['I am going outside again.',
 'I have returned home and am going to take a bath',
 'I am going outside.']

In [ ]:
memory_handler.update_duplicate_records()

KeyboardInterrupt: 

In [ ]:
memory_handler.fetch_records()

In [ ]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma

from langchain.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)


# # Examples of a pretend task of creating antonyms.
# examples = [
#     {"input": "happy", "output": "sad"},
#     {"input": "tall", "output": "short"},
#     {"input": "energetic", "output": "lethargic"},
#     {"input": "sunny", "output": "gloomy"},
#     {"input": "windy", "output": "calm"},
# ]

In [ ]:
input = memory_handler.fetch_records()
print(input)

['What are top DeepLearning algorithums used in Medical field?', 'Can you provide me context about the earlier conversation we had about algorithums?', 'Can you provide me context about the earlier conversation we had about Medical?', 'And what about in medical field?', 'How AI will extract information from the conversation?', 'Which Algorithm is will be used to detect abnormality in analyzing medical images', 'Which Algorithm is will be used to detect abnormality in analyzing medical images', 'Which Algorithm is will be used to detect abnormality in analyzing medical images', 'Which Algorithm is will be used to detect abnormality in analyzing medical images', 'And specifically, how will ai will be used to do that?', 'How AI can be used in healthcare?', 'Can you tell me about yourself?', 'Hello, how are you?']


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

emb_list = embeddings.embed_documents(input)

In [ ]:


def calculate_similarity(emb1, emb2):
    return 1 - distance.cosine(emb1, emb2)

In [ ]:
def duplicate_records(self, records, emb_list):
    records_list = records
    embeddings_list = emb_list
    duplicates = []
    checked_pairs = set()  

    for i in range(len(records_list)):
        for j in range(i + 1, len(records_list)):  
            pair = tuple(sorted([i, j]))  
            if pair not in checked_pairs:
                similarity_score = calculate_similarity(embeddings_list[i], embeddings_list[j])
                if similarity_score > 0.9:
                    duplicates.append([records_list[i], records_list[j]])
                checked_pairs.add(pair)  

    return duplicates

In [ ]:
duplicates = duplicate_records(memory_handler, records=input, emb_list=emb_list)


In [ ]:
inp_2= ['I am going outside.','I have returned home and am going to take a bath','I am going outside again.']

emb_list_2 = embeddings.embed_documents(inp_2)

In [ ]:
duplicates_2 = duplicate_records(memory_handler, records=inp_2, emb_list=emb_list_2)



In [ ]:
duplicates_2

[['I am going outside.', 'I am going outside again.']]

In [ ]:
inp_3= ['I am going outside.','I have returned home and am going to take a bath','I am going outside again.','I came back home.', 'I am going out for shopping.']

emb_list_3 = embeddings.embed_documents(inp_3)


In [ ]:
duplicates_3 = duplicate_records(memory_handler, records=inp_3, emb_list=emb_list_3)

duplicates_3

[['I am going outside.', 'I am going outside again.'],
 ['I am going outside.', 'I am going out for shopping.']]